# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    import numpy as np
    
    #list of different patterns
    #pattern-01: 03/25/93
    df_pattern_01 = r'(?P<date1>\d{1,2}[-/]\d{1,2}[-/]\d{2,4})'
    
    #pattern-02: February 18, 1981
    df_pattern_02 = r'(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,.-/ ]? (?P<date2>\d{1,2})[a-z]{0,2}[,.-/ ]? (?P<year2>\d{4})'
    #r'(?P<date2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.-/ ]? \d{1,2}[a-z]{0,2}[,.-/ ]? \d{2,4})' 
    #pattern-03: 24 Jan 2001
    df_pattern_03 =  r'(?P<date3>\d{2})\s(?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)\s(?P<year3>\d{4})'
    
    #pattern-04: September 1985 / July, 1990
    df_pattern_04 = r'(?P<date4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.-/ ]? \d{2,4})'
    
    #pattern-05: 8/1974 
    df_pattern_05= r'(?P<date5>\d{1,2}[-/]\d{4})'
    
    #pattern-06: only year 2008
    df_pattern_06 = r'(?P<date6>\d{4})'
    
    
    

    data2 = df.str.extract(df_pattern_02) 
    #data2[~data2['month'].isna()]

    month_dict = {
        'Jan':'1',
        'Feb':'2',
        'Mar':'3',    
        'Apr':'4',    
        'May':'5',  
        'Jun':'6',   
        'Jul':'7',   
        'Aug':'8',
        'Sep':'9',
        'Oct':'10',
        'Nov':'11',
        'Dec':'12'
        }

    #only return 1st three characters of each month
    data2['months'] = data2['month2'].str[:3].map(month_dict)
    
    #data2['months'] = data2['months'].map(month_dict)
    
    
    #data2[~data2['months'].isna()]
    #rearrange the date format dd-mm-yyyy
    data2['date2'] = data2[['months', 'date2', 'year2']].astype(str).agg('/'.join, axis=1)
    
    #replacing the value with NaN
    data2 = data2.replace('nan/nan/nan',np.NaN)
    new_data2 = data2['date2']
    
    #new_data2
    #data5 = df.str.extract(df_pattern_05) 
    #data5['date5'].notnull().sum()
    
    #to find 06 May 1993
    
    data3 = df.str.extract(df_pattern_03)
    #only return 1st three characters of each month
    data3['months'] = data3['month3'].str[:3].map(month_dict)
    data3['date3'] = data3[['months', 'date3', 'year3']].astype(str).agg('/'.join, axis=1)

    #replacing the value with NaN
    data3 = data3.replace('nan/nan/nan',np.NaN)
    new_data3 = data3['date3']
    
    #matching different types of patterns and store it to different columns
    patterns = [df_pattern_01, df_pattern_04, df_pattern_05, df_pattern_06]
    data= pd.concat([df.str.extract(p) for p in patterns], axis =1)
    dates = data.join(new_data2)
    dates = dates.join(new_data3)
    
    #date
    #name correction
    dates['date4'] = data['date4'].replace('Decemeber 1978','December 1978')
    
     #to add new col with NaN value
    dates['date'] = pd.Series(dtype='int')
    
    
    columns = ['date1', 'date2', 'date3', 'date4', 'date5', 'date6']
    for col in columns:
        dates['date'] = dates['date'].fillna(dates[col])
    
    #apply datetime on overall data to get standardize all rows with same (yyyy-mm-dd) format
    dates = dates.apply(pd.to_datetime, errors='coerce')
    #data[data['date']>="2021-12-12"]
    #dates
    
    #null value appears due to value conversion and now being replaced
    dates['date'] = dates['date'].fillna(dates['date6'])
    
    #to change mistaken outcome 2070/2071 t0 1970/71
    dates['date'] = np.where(dates['date'].dt.year > 2023, dates['date'] - pd.offsets.DateOffset(years=100), dates['date'])
    
    #sorting the date and get the index order
    sorting = pd.Series(dates["date"].reset_index(drop=False).sort_values(["date", "index"]).index)
   
    return sorting
    raise NotImplementedError()
    return order
    

# Index and Sorting

pd.Series(dates["date"].reset_index(drop=False).sort_values(["date", "index"]).index)

In [32]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

<h2>Kendall Tau: 0.9999999999999999</h2>

<h2>To check sorting is okay or not</h2>

In [4]:

import numpy as np
s = date_sorter()
test = pd.DataFrame({'correct':
[6695, 14428, 16742, 9275, 12290, 14654, 9421, 10185, 11464, 16491,
 11797, 14036, 15459, 9412, 13069, 10400, 10498, 14322, 13274, 11001,
 11383, 11910, 10977, 9692, 10199, 10187, 15456, 13491, 9186, 13646,
 11142, 13724, 10994, 12905, 15968, 16648, 13966, 14607, 16932, 14622,
 17942, 18220, 17818, 18305, 19633, 12522, 13978, 18445, 20156, 14797],
'learner':[
(s.iloc[10*i:(i+1)*10].values * np.array(range(1,11))).sum() for i in range(50)]},
index=range(0,500,10)).assign(agree=lambda x:x['correct']==x['learner'])

test

,correct,learner,agree
0,6695,6695,True
10,14428,14428,True
20,16742,16742,True
30,9275,9275,True
40,12290,12290,True
50,14654,14654,True
60,9421,9421,True
70,10185,10185,True
80,11464,11464,True
90,16491,16491,True


<h2>Different Pattern outcome</h2>

In [18]:
#pattern-01: 03/25/93
df_pattern_01 = r'(?P<date1>\d{1,2}[-/]\d{1,2}[-/]\d{2,4})'

#pattern-02: February 18, 1981
df_pattern_02 = r'(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,.-/ ]? (?P<date2>\d{1,2})[a-z]{0,2}[,.-/ ]? (?P<year2>\d{4})'
#r'(?P<date2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.-/ ]? \d{1,2}[a-z]{0,2}[,.-/ ]? \d{2,4})' 
#pattern-03: 24 Jan 2001
df_pattern_03 =  r'(?P<date3>\d{2})\s(?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)\s(?P<year3>\d{4})'

#pattern-04: September 1985 / July, 1990
df_pattern_04 = r'(?P<date4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.-/ ]? \d{2,4})'

#pattern-05: 8/1974 
df_pattern_05= r'(?P<date5>\d{1,2}[-/]\d{4})'

#pattern-06: only year 2008
df_pattern_06 = r'(?P<date6>\d{4})'

In [19]:
df_pattern_01 = r'(?P<date1>\d{1,2}[-/]\d{1,2}[-/]\d{2,4})'
def pattern(insert_pattern):
    data = df.str.extract(insert_pattern) 
    return data

In [26]:
pattern(df_pattern_01).dropna()[0:5]

,date1
0,03/25/93
1,6/18/85
2,7/8/71
3,9/27/75
4,2/6/96


In [25]:
pattern(df_pattern_02).dropna()[0:5]

,month2,date2,year2
194,April,11,1990
195,May,30,2001
196,Feb,18,1994
197,February,18,1981
198,October,11,2013


In [27]:
pattern(df_pattern_03).dropna()[0:5]

,date3,month3,year3
125,24,Jan,2001
126,10,Sep,2004
127,26,May,1982
128,28,June,2002
129,06,May,1972


In [28]:
pattern(df_pattern_04).dropna()[0:5]

,date4
125,Jan 2001
126,Sep 2004
127,May 1982
128,June 2002
129,May 1972


In [29]:
pattern(df_pattern_05).dropna()[0:5]

,date5
14,24/1990
15,25/2011
17,13/1976
24,25/1984
30,31/1985


In [30]:
pattern(df_pattern_06).dropna()[0:5]

,date6
7,9080
14,1990
15,2011
17,1976
22,9079


# Dataset

In [36]:
    
import numpy as np

#list of different patterns
#pattern-01: 03/25/93
df_pattern_01 = r'(?P<date1>\d{1,2}[-/]\d{1,2}[-/]\d{2,4})'

#pattern-02: February 18, 1981
df_pattern_02 = r'(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,.-/ ]? (?P<date2>\d{1,2})[a-z]{0,2}[,.-/ ]? (?P<year2>\d{4})'
#r'(?P<date2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.-/ ]? \d{1,2}[a-z]{0,2}[,.-/ ]? \d{2,4})' 
#pattern-03: 24 Jan 2001
df_pattern_03 =  r'(?P<date3>\d{2})\s(?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)\s(?P<year3>\d{4})'

#pattern-04: September 1985 / July, 1990
df_pattern_04 = r'(?P<date4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.-/ ]? \d{2,4})'

#pattern-05: 8/1974 
df_pattern_05= r'(?P<date5>\d{1,2}[-/]\d{4})'

#pattern-06: only year 2008
df_pattern_06 = r'(?P<date6>\d{4})'



data2 = df.str.extract(df_pattern_02) 
#data2[~data2['month'].isna()]
month_dict = {
    'Jan':'1',
    'Feb':'2',
    'Mar':'3',    
    'Apr':'4',    
    'May':'5',  
    'Jun':'6',   
    'Jul':'7',   
    'Aug':'8',
    'Sep':'9',
    'Oct':'10',
    'Nov':'11',
    'Dec':'12'
    }
#only return 1st three characters of each month
data2['months'] = data2['month2'].str[:3].map(month_dict)

#data2['months'] = data2['months'].map(month_dict)


#data2[~data2['months'].isna()]
#rearrange the date format dd-mm-yyyy
data2['date2'] = data2[['months', 'date2', 'year2']].astype(str).agg('/'.join, axis=1)
    
#replacing the value with NaN
data2 = data2.replace('nan/nan/nan',np.NaN)
new_data2 = data2['date2']

#new_data2
#data5 = df.str.extract(df_pattern_05) 
#data5['date5'].notnull().sum()

#to find 06 May 1993

data3 = df.str.extract(df_pattern_03)
#only return 1st three characters of each month
data3['months'] = data3['month3'].str[:3].map(month_dict)
data3['date3'] = data3[['months', 'date3', 'year3']].astype(str).agg('/'.join, axis=1)
#replacing the value with NaN
data3 = data3.replace('nan/nan/nan',np.NaN)
new_data3 = data3['date3']

#matching different types of patterns and store it to different columns
patterns = [df_pattern_01, df_pattern_04, df_pattern_05, df_pattern_06]
data= pd.concat([df.str.extract(p) for p in patterns], axis =1)
dates = data.join(new_data2)
dates = dates.join(new_data3)

#date
#name correction
dates['date4'] = data['date4'].replace('Decemeber 1978','December 1978')

 #to add new col with NaN value
dates['date'] = pd.Series(dtype='int')


columns = ['date1', 'date2', 'date3', 'date4', 'date5', 'date6']
for col in columns:
    dates['date'] = dates['date'].fillna(dates[col])

dates[~dates['date'].isna()]

,date1,date4,date5,date6,date2,date3,date
0,03/25/93,NaN,NaN,NaN,NaN,NaN,03/25/93
1,6/18/85,NaN,NaN,NaN,NaN,NaN,6/18/85
2,7/8/71,NaN,NaN,NaN,NaN,NaN,7/8/71
3,9/27/75,NaN,NaN,NaN,NaN,NaN,9/27/75
4,2/6/96,NaN,NaN,NaN,NaN,NaN,2/6/96
...,...,...,...,...,...,...,...
495,NaN,NaN,NaN,1979,NaN,NaN,1979
496,NaN,NaN,NaN,2006,NaN,NaN,2006
497,NaN,NaN,NaN,2008,NaN,NaN,2008
498,NaN,NaN,NaN,2005,NaN,NaN,2005
